# PC1 - Grupo 2 (Julia)

#### Integrantes
- GARCIA RODRIGUEZ, EMILIO ALONSO
- PADILLA AQUISE, ALESSANDRO PIERO
- RIEGA NUÑEZ, GABRIEL ANTONIO FERMIN
- SALAMANCA FERNANDEZ, LUCAS PABLO
- SILVA ANDUJAR, NICOLAS

## 1. Data analysis

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.8.2 on the current Colab Runtime...
2024-09-07 01:34:38 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.8/julia-1.8.2-linux-x86_64.tar.gz [135859273/135859273] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.8

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


In [1]:
versioninfo()

Julia Version 1.8.2
Commit 36034abf260 (2022-09-29 15:21 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, broadwell)
  Threads: 2 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = 2


In [2]:
using Pkg
Pkg.add("Plots")
Pkg.add("Random")
Pkg.add("GLM")
Pkg.add("DataFrames")
Pkg.add("LaTeXStrings")
Pkg.add("Statistics")
Pkg.add("StatsBase")
Pkg.add("CSV")
Pkg.add("CategoricalArrays")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed JpegTurbo_jll ──────────────── v3.0.3+0
   Installed GR_jll ─────────────────────── v0.73.7+0
   Installed x265_jll ───────────────────── v3.5.0+0
   Installed libfdk_aac_jll ─────────────── v2.0.3+0
   Installed Libmount_jll ───────────────── v2.40.1+0
   Installed libdecor_jll ───────────────── v0.2.2+0
   Installed LERC_jll ───────────────────── v3.0.0+1
   Installed LoggingExtras ──────────────── v1.0.3
   Installed Unitful ────────────────────── v1.21.0
   Installed Opus_jll ───────────────────── v1.3.3+0
   Installed Xorg_xkbcomp_jll ───────────── v1.4.6+0
   Installed RelocatableFolders ─────────── v1.0.1
   Installed Measures ───────────────────── v0.3.2
   Installed ConcurrentUtilities ────────── v2.4.2
   Installed Contour ────────────────────── v0.6.3
   Installed Grisu ──────────────────────── v1.0.2
   Installed Xorg_xcb_util_wm_jll ───────── v0.4.1+1
   Installed Xorg

In [3]:
using Plots
using Random
using GLM
using DataFrames
using LaTeXStrings
using Statistics
using StatsBase

1.Import the data set. Make sure the column names are imported as intended variable

In [74]:
using CSV

data = CSV.read("wage2015_subsample_inference.csv", DataFrame)

first(data,5)

Row,rownames,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Int64
1,10,9.61538,2.26336,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,3600.0,11,8370.0,18
2,12,48.0769,3.8728,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,3050.0,10,5070.0,9
3,15,11.0577,2.40313,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,3.24,5.832,10.4976,6260.0,19,770.0,4
4,18,13.9423,2.63493,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,25.0,6.25,15.625,39.0625,420.0,1,6990.0,12
5,19,28.8462,3.36198,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,2015.0,6,9470.0,22


2.Are there missing values? Display the number of missings (if any) of each variable in the data set.

In [75]:
# Find number of missing values for each column
missing_values = map(col -> sum(ismissing, col), eachcol(data))

# Handle missing values a dataframe
missing_df = DataFrame(Column = names(data), MissingCount = missing_values)

println(missing_df)

# Check for missing values
if all(count -> count == 0, missing_values)
    println("There are no missing values")
else
    println("There are missing values")
end


21×2 DataFrame
 Row │ Column    MissingCount 
     │ String    Int64        
─────┼────────────────────────
   1 │ rownames             0
   2 │ wage                 0
   3 │ lwage                0
   4 │ sex                  0
   5 │ shs                  0
   6 │ hsg                  0
   7 │ scl                  0
   8 │ clg                  0
   9 │ ad                   0
  10 │ mw                   0
  11 │ so                   0
  12 │ we                   0
  13 │ ne                   0
  14 │ exp1                 0
  15 │ exp2                 0
  16 │ exp3                 0
  17 │ exp4                 0
  18 │ occ                  0
  19 │ occ2                 0
  20 │ ind                  0
  21 │ ind2                 0
There are no missing values


3.Report descriptive statistics of the variables (mean, standard deviation, percentiles, etc.). Interpret your results.

In [76]:
describe(data)


Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,DataType
1,rownames,15636.3,10,15260.0,32643,0,Int64
2,wage,23.4104,3.02198,19.2308,528.846,0,Float64
3,lwage,2.97079,1.10591,2.95651,6.2707,0,Float64
4,sex,0.444466,0.0,0.0,1.0,0,Float64
5,shs,0.023301,0.0,0.0,1.0,0,Float64
6,hsg,0.243883,0.0,0.0,1.0,0,Float64
7,scl,0.278058,0.0,0.0,1.0,0,Float64
8,clg,0.31767,0.0,0.0,1.0,0,Float64
9,ad,0.137087,0.0,0.0,1.0,0,Float64


In [77]:
function calculate_quartiles(df)
    quartiles = DataFrame(
        Variable = names(df),
        Q1 = quantile.(eachcol(df), Ref(0.25)),
        Q2 = quantile.(eachcol(df), Ref(0.50)),  # Median
        Q3 = quantile.(eachcol(df), Ref(0.75))
    )
    return quartiles
end

quartiles_summary = calculate_quartiles(data)

println(quartiles_summary)

21×4 DataFrame
 Row │ Variable  Q1          Q2           Q3          
     │ String    Float64     Float64      Float64     
─────┼────────────────────────────────────────────────
   1 │ rownames  7132.5      15260.0      24337.8
   2 │ wage        13.4615      19.2308      27.7778
   3 │ lwage        2.59984      2.95651      3.32424
   4 │ sex          0.0          0.0          1.0
   5 │ shs          0.0          0.0          0.0
   6 │ hsg          0.0          0.0          0.0
   7 │ scl          0.0          0.0          1.0
   8 │ clg          0.0          0.0          1.0
   9 │ ad           0.0          0.0          0.0
  10 │ mw           0.0          0.0          1.0
  11 │ so           0.0          0.0          1.0
  12 │ we           0.0          0.0          0.0
  13 │ ne           0.0          0.0          0.0
  14 │ exp1         5.0         10.0         21.0
  15 │ exp2         0.25         1.0          4.41
  16 │ exp3         0.125        1.0          9.261
  17 │ exp

In [78]:
#Std deviation

# Function that calculates the std deviation for each column
function calculate_standard_deviation(df::DataFrame)
    std_values = Dict{Symbol, Any}()

    for col in names(df)
        # Change column name to symbol
        col_sym = Symbol(col)

        # Calculate std deviation
        std_val = std(df[!, col_sym])
        std_values[col_sym] = std_val

    end
    return std_values
end

std_dict = calculate_standard_deviation(data)
println("Standard deviation for each variable:")
println(std_dict)

Standard deviation for each variable:
Dict{Symbol, Any}(:so => 0.4567605898390435, :hsg => 0.42946495731602635, :mw => 0.438464105331998, :wage => 21.003016115654056, :exp1 => 10.609465081743178, :occ => 11874.356080073781, :sex => 0.49695465704835745, :ind2 => 5.701019018248305, :ne => 0.419432173656693, :rownames => 9700.744394538977, :occ2 => 6.966684461108375, :lwage => 0.5703847829926473, :we => 0.41163462287504776, :exp2 => 4.000904430517199, :exp3 => 14.488961873778786, :exp4 => 53.53022497424143, :shs => 0.15087231530138967, :scl => 0.4480857588192327, :ad => 0.3439729691578773, :ind => 5333.443992032589, :clg => 0.4656155413671081)


#### wage:
The average wage in the sample is 23.41, with a standard deviation of 21.03, reflecting significant variability in earnings. Wages range from a minimum of 3.02 to a maximum of 528.85, indicating a broad spectrum of income levels. The bottom 25th percentile earn less than 13.46, while the median wage is 19.23, and the top 25% earn more than 27.78. The fact that the median is 19.23 suggests the presence of a number of extremely high wage values, which likely skews the distribution.

#### lwage:
The logarithm of wage has a mean of 2.97 and a standard deviation of 0.57, suggesting that the distribution of wages in its logarithmic scale is more symmetric than in its original form. The logarithmic wage values range from a minimum of 1.11 to a maximum of 6.27, with the top 25% of the sample having a log wage greater than 3.32. Also, it has a median of 2.96, that suggest a lower presence of extremely high or low wages in the sample.

#### sex:
Although sex is a dummy variable, we can see that 44% of the sample are females, while the mode is "male." The standard deviation of 0.5 suggests an almost balanced distribution between males and females.

#### shs:
The "some high school" variable shows that only 2.3% of the sample have some incomplete high school education, which represents a small fraction of the total population.

#### hsg:
About 24% of the sample has graduated from high school, according to the mean of this variable. The standard deviation of 0.43 reflects moderate diversity in the education levels within the sample, with a significant portion not having attained a college degree.

#### scl:
Approximately 28% of the respondents have attended some college without necessarily graduating, as indicated by the mean. This suggests that a considerable segment of the population has received some tertiary education, although they may not have completed their studies.

#### clg:
The data shows that 32% of the sample holds a college degree, as reflected in the mean of this variable. This implies that nearly one-third of the sample has completed a college education, which is significant for evaluating human capital.

#### ad:
About 14% of the individuals in the sample have an advanced degree (master's or doctorate), according to the mean of this variable. This relatively low percentage highlights that advanced education is less common compared to basic college education.

#### mw:
The Midwest region represents 26% of the sample, as indicated by the mean of this variable. This suggests that this region has significant representation within the dataset.

#### so:
The South is the most represented region, with 30% of the sample residing in this area, as shown by the mean of this variable. The higher presence of the South may reflect certain geographical trends in the sample.

#### we:
The West region accounts for 22% of the sample, according to the mean. This proportion suggests a relatively equitable distribution among the country's regions, although with less representation compared to the South.

#### ne:
The Northeast region represents 23% of the sample, as indicated by the mean of this variable. This proportion, similar to the other regions, shows a relatively balanced geographical distribution in the dataset.

#### exp1:
The average work experience is 13.76 years, with a standard deviation of 10.61, indicating high variability in the number of years worked among the individuals in the sample. The median is 10 years, meaning that half of the sample has 10 years or less of work experience, while the maximum is 47 years. Also, the mode of work experience is 5.

#### exp2:
The first polynomial of work experience has a mean of 3.02 years, with a standard deviation of 4.00. This suggests that, for certain specific aspects of work experience, most of the sample has relatively low experience.

#### exp3:
The second polynomial of work experience, exp3, has a mean of 8.24 years and a standard deviation of 14.49, indicating a wide variability.

#### exp4:
The third polynomial of work experience, exp4, shows a significantly higher mean of 25.12 years, with a standard deviation of 53.53, again reflecting a high variability.

4.How many women with a college graduate degree (clg) or above have a wage corresponding to the 25% richest of the sample? Report the dataframe corresponding to this conditions and the result.

In [79]:
# Calculate 75th percentile based on wage
percentile_75_wage = quantile(skipmissing(data.wage), 0.75)

# Filter educated women with a salary belonging to the top 25%
rich_ed_women = filter(row -> (row.sex == 1) &&
                             ((row.clg == 1) || (row.ad == 1)) &&
                             (row.wage >= percentile_75_wage), data)

rich_ed_women

Row,rownames,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Int64
1,19,28.8462,3.36198,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,2015.0,6,9470.0,22
2,191,42.3077,3.74497,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4700.0,16,4795.0,9
3,232,41.2088,3.71865,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,11.0,1.21,1.331,1.4641,4700.0,16,5790.0,9
4,319,100.0,4.60517,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,27.0,7.29,19.683,53.1441,850.0,2,6970.0,12
5,563,33.6538,3.51613,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,4.0,0.16,0.064,0.0256,710.0,2,7390.0,14
6,842,36.0577,3.58512,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,23.0,5.29,12.167,27.9841,420.0,1,9170.0,21
7,865,31.25,3.44202,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,15.0,2.25,3.375,5.0625,3255.0,10,8190.0,18
8,892,28.8462,3.36198,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,9.0,0.81,0.729,0.6561,540.0,2,6990.0,12
9,1052,48.0769,3.8728,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,24.0,5.76,13.824,33.1776,4850.0,16,3390.0,6


In [80]:
println("Number of women with a college graduate degree or above in the top 25% richest:",size(rich_ed_women,1))

Number of women with a college graduate degree or above in the top 25% richest:419


5.How many men with a high school graduate degree (hsg) or below have a wage corresponding to the 25% richest of the sample? Report the dataframe corresponding to this conditions and the result.

In [81]:
# Filter men with high school or below
rich_uned_men = filter(row-> (row.sex == 0) &&
                            ((row.hsg == 1) || (row.shs == 1)) &&
                            (row.wage >= percentile_75_wage), data)

rich_uned_men

Row,rownames,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Int64
1,113,27.8846,3.32808,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,16.0,2.56,4.096,6.5536,6355.0,19,770.0,4
2,276,28.8462,3.36198,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,41.0,16.81,68.921,282.576,9620.0,22,4970.0,9
3,467,28.8462,3.36198,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,33.0,10.89,35.937,118.592,8220.0,21,3680.0,6
4,858,28.8462,3.36198,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,40.0,16.0,64.0,256.0,4700.0,16,4870.0,9
5,876,29.7143,3.39163,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0,0.64,0.512,0.4096,4040.0,13,8680.0,20
6,1065,27.7778,3.32424,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,33.0,10.89,35.937,118.592,9130.0,22,6380.0,10
7,1359,35.5769,3.5717,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0,0.64,0.512,0.4096,6440.0,19,770.0,4
8,1870,36.3248,3.5925,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,30.0,9.0,27.0,81.0,6260.0,19,770.0,4
9,2194,43.2692,3.76744,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,29.0,8.41,24.389,70.7281,7410.0,20,570.0,3


In [82]:
println("Number of men with a high school degree or below in the top 25% richest:", size(rich_uned_men,1))

Number of men with a high school degree or below in the top 25% richest:118


6.Create two dataframes. One containing only the log(wage) and the other containig every variable of the data set but the wage related variable.


In [83]:
# DataFrame for'lwage'
log_w = data[:, [:lwage]]

# DataFrame for the rest of the columns except 'rownames' 'wage' y 'lwage'
regs = select(data, Not([:rownames, :wage, :lwage]))

display(log_w)
display(regs)


Row,lwage
,Float64
1,2.26336
2,3.8728
3,2.40313
4,2.63493
5,3.36198
6,2.46222
7,2.95651
8,2.95651
9,2.48491


Row,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Int64
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,3600.0,11,8370.0,18
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,3050.0,10,5070.0,9
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,3.24,5.832,10.4976,6260.0,19,770.0,4
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,25.0,6.25,15.625,39.0625,420.0,1,6990.0,12
5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,2015.0,6,9470.0,22
6,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.01,0.001,0.0001,1650.0,5,7460.0,14
7,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,42.0,17.64,74.088,311.17,5120.0,17,7280.0,14
8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,37.0,13.69,50.653,187.416,5240.0,17,5680.0,9
9,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,4040.0,13,8590.0,19


## 2. Data Wrangling

7.Make an array for our y variable, which will be the logarithm of wage (lwage column).

In [84]:
#This has aleady been solved in question 6, as there is no need to create a specific array in Julia

size(log_w)

(5150, 1)

8.Make three new arrays for our predictors:

8.1. The basic model will have the columns sex hsg scl clg ad so we ne exp1 occ2 ind2. Make sure to convert occ2 and ind2 to dummies and to drop the first dummy value to prevent multicolinearity. This means:

In [85]:
# Categories for occ2
sort(unique(data.occ2))


22-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
 15
 16
 17
 18
 19
 20
 21
 22

In [86]:
# Make occ2 a dummy:
basic_model = regs
for i in 2:22
    col_name = Symbol("occ2_$i")
    basic_model = transform(basic_model, :occ2 => ByRow(x -> x == i ? 1 : 0) => col_name)
end

println(basic_model)


5150×39 DataFrame
  Row │ sex      shs      hsg      scl      clg      ad       mw       so       we       ne       exp1     exp2     exp3        exp4         occ       occ2   ind       ind2   occ2_2  occ2_3  occ2_4  occ2_5  occ2_6  occ2_7  occ2_8  occ2_9  occ2_10  occ2_11  occ2_12  occ2_13  occ2_14  occ2_15  occ2_16  occ2_17  occ2_18  occ2_19  occ2_20  occ2_21  occ2_22 
      │ Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64     Float64      Float64   Int64  Float64   Int64  Int64   Int64   Int64   Int64   Int64   Int64   Int64   Int64   Int64    Int64    Int64    Int64    Int64    Int64    Int64    Int64    Int64    Int64    Int64    Int64    Int64   
──────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Excessive output truncated after 524288 bytes.

In [87]:
# Categories for ind2
sort(unique(data.ind2))

21-element Vector{Int64}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
 15
 16
 17
 18
 19
 20
 21
 22

In [88]:
# Make occ2 a dummy:
for i in 2:21
    col_name = Symbol("ind2_$i")
    basic_model = transform(basic_model, :ind2 => ByRow(x -> x == i ? 1 : 0) => col_name)
end

basic_model
ncol(basic_model)

59

In [89]:
#Basic Model

select!(basic_model, Not(["shs", "mw", "occ", "ind", "exp2", "exp3", "exp4"]))
basic_model

Row,sex,hsg,scl,clg,ad,so,we,ne,exp1,occ2,ind2,occ2_2,occ2_3,occ2_4,occ2_5,occ2_6,occ2_7,occ2_8,occ2_9,occ2_10,occ2_11,occ2_12,occ2_13,occ2_14,occ2_15,occ2_16,occ2_17,occ2_18,occ2_19,occ2_20,occ2_21,occ2_22,ind2_2,ind2_3,ind2_4,ind2_5,ind2_6,ind2_7,ind2_8,ind2_9,ind2_10,ind2_11,ind2_12,ind2_13,ind2_14,ind2_15,ind2_16,ind2_17,ind2_18,ind2_19,ind2_20,ind2_21
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,7.0,11,18,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,31.0,10,9,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,19,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,25.0,1,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
5,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,22.0,6,22,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,5,14,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
7,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,42.0,17,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
8,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,37.0,17,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
9,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,31.0,13,19,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


8.2. The flexible model will have the same columns, and will also include polynomials for experience (exp2 exp3 exp4), as well as the interactions between all experience variables and other variables except for sex. This means:

In [90]:
#Flexible Model

flex_model = basic_model

insertcols!(flex_model, :exp2 => regs.exp2, :exp3 => regs.exp3, :exp4 => regs.exp4)

flex_model



Row,sex,hsg,scl,clg,ad,so,we,ne,exp1,occ2,ind2,occ2_2,occ2_3,occ2_4,occ2_5,occ2_6,occ2_7,occ2_8,occ2_9,occ2_10,occ2_11,occ2_12,occ2_13,occ2_14,occ2_15,occ2_16,occ2_17,occ2_18,occ2_19,occ2_20,occ2_21,occ2_22,ind2_2,ind2_3,ind2_4,ind2_5,ind2_6,ind2_7,ind2_8,ind2_9,ind2_10,ind2_11,ind2_12,ind2_13,ind2_14,ind2_15,ind2_16,ind2_17,ind2_18,ind2_19,ind2_20,ind2_21,exp2,exp3,exp4
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,7.0,11,18,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.49,0.343,0.2401
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,31.0,10,9,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,9.61,29.791,92.3521
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,19,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.24,5.832,10.4976
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,25.0,1,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,6.25,15.625,39.0625
5,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,22.0,6,22,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.84,10.648,23.4256
6,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,5,14,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.01,0.001,0.0001
7,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,42.0,17,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,17.64,74.088,311.17
8,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,37.0,17,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,13.69,50.653,187.416
9,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,31.0,13,19,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,9.61,29.791,92.3521


In [91]:
# Flexible model - interactions

exp_vars = [:exp1, :exp2, :exp3, :exp4]

other_vars = [
    :hsg, :scl, :clg, :ad, :so, :we, :ne,
    :occ2, :ind2,
    :occ2_2, :occ2_3, :occ2_4, :occ2_5, :occ2_6, :occ2_7, :occ2_8, :occ2_9,
    :occ2_10, :occ2_11, :occ2_12, :occ2_13, :occ2_14, :occ2_15, :occ2_16,
    :occ2_17, :occ2_18, :occ2_19, :occ2_20, :occ2_21, :occ2_22,
    :ind2_2, :ind2_3, :ind2_4, :ind2_5, :ind2_6, :ind2_7, :ind2_8, :ind2_9,
    :ind2_10, :ind2_11, :ind2_12, :ind2_13, :ind2_14, :ind2_15, :ind2_16,
    :ind2_17, :ind2_18, :ind2_19, :ind2_20, :ind2_21
]

# Create interactions
for exp in exp_vars
    for var in other_vars
        # New column name
        interaction_name = Symbol("$(exp)_x_$(var)")

        flex_model[!, interaction_name] = flex_model[!, exp] .* flex_model[!, var]
    end
end

flex_model

Row,sex,hsg,scl,clg,ad,so,we,ne,exp1,occ2,ind2,occ2_2,occ2_3,occ2_4,occ2_5,occ2_6,occ2_7,occ2_8,occ2_9,occ2_10,occ2_11,occ2_12,occ2_13,occ2_14,occ2_15,occ2_16,occ2_17,occ2_18,occ2_19,occ2_20,occ2_21,occ2_22,ind2_2,ind2_3,ind2_4,ind2_5,ind2_6,ind2_7,ind2_8,ind2_9,ind2_10,ind2_11,ind2_12,ind2_13,ind2_14,ind2_15,ind2_16,ind2_17,ind2_18,ind2_19,ind2_20,ind2_21,exp2,exp3,exp4,exp1_x_hsg,exp1_x_scl,exp1_x_clg,exp1_x_ad,exp1_x_so,exp1_x_we,exp1_x_ne,exp1_x_occ2,exp1_x_ind2,exp1_x_occ2_2,exp1_x_occ2_3,exp1_x_occ2_4,exp1_x_occ2_5,exp1_x_occ2_6,exp1_x_occ2_7,exp1_x_occ2_8,exp1_x_occ2_9,exp1_x_occ2_10,exp1_x_occ2_11,exp1_x_occ2_12,exp1_x_occ2_13,exp1_x_occ2_14,exp1_x_occ2_15,exp1_x_occ2_16,exp1_x_occ2_17,exp1_x_occ2_18,exp1_x_occ2_19,exp1_x_occ2_20,exp1_x_occ2_21,exp1_x_occ2_22,exp1_x_ind2_2,exp1_x_ind2_3,exp1_x_ind2_4,exp1_x_ind2_5,exp1_x_ind2_6,exp1_x_ind2_7,exp1_x_ind2_8,exp1_x_ind2_9,exp1_x_ind2_10,exp1_x_ind2_11,exp1_x_ind2_12,exp1_x_ind2_13,exp1_x_ind2_14,exp1_x_ind2_15,exp1_x_ind2_16,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,7.0,11,18,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.49,0.343,0.2401,0.0,0.0,7.0,0.0,0.0,0.0,7.0,77.0,126.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,31.0,10,9,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,9.61,29.791,92.3521,0.0,0.0,31.0,0.0,0.0,0.0,31.0,310.0,279.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,19,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.24,5.832,10.4976,18.0,0.0,0.0,0.0,0.0,0.0,18.0,342.0,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,25.0,1,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,6.25,15.625,39.0625,0.0,0.0,0.0,25.0,0.0,0.0,25.0,25.0,300.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,⋯
5,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,22.0,6,22,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.84,10.648,23.4256,0.0,0.0,22.0,0.0,0.0,0.0,22.0,132.0,484.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
6,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,5,14,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.01,0.001,0.0001,0.0,0.0,1.0,0.0,0.0,0.0,1.0,5.0,14.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,⋯
7,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,42.0,17,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,17.64,74.088,311.17,42.0,0.0,0.0,0.0,0.0,0.0,42.0,714.0,588.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

8.3. The extra-flexible model will include all two-way interactions between variables, except for sex.

In [92]:
# Extra flexible model

extra_flex_model = flex_model


# Variables without "sex"
vars_to_interact = [
    :exp1, :exp2, :exp3, :exp4, :hsg, :scl, :clg, :ad, :so, :we, :ne,
    :occ2, :ind2, :occ2_2, :occ2_3, :occ2_4, :occ2_5, :occ2_6, :occ2_7, :occ2_8,
    :occ2_9, :occ2_10, :occ2_11, :occ2_12, :occ2_13, :occ2_14, :occ2_15, :occ2_16,
    :occ2_17, :occ2_18, :occ2_19, :occ2_20, :occ2_21, :occ2_22,
    :ind2_2, :ind2_3, :ind2_4, :ind2_5, :ind2_6, :ind2_7, :ind2_8, :ind2_9,
    :ind2_10, :ind2_11, :ind2_12, :ind2_13, :ind2_14, :ind2_15, :ind2_16,
    :ind2_17, :ind2_18, :ind2_19, :ind2_20, :ind2_21
]

# Create two way interactions
for i in 1:length(vars_to_interact)
    for j in (i+1):length(vars_to_interact)  # begin from `i+1` to avoid duplicates
        var1 = vars_to_interact[i]
        var2 = vars_to_interact[j]

        # Name of new interactions
        interaction_name = Symbol("$(var1)_x_$(var2)")

        # Create interactions
        extra_flex_model[!, interaction_name] = extra_flex_model[!, var1] .* extra_flex_model[!, var2]
    end
end

extra_flex_model


Row,sex,hsg,scl,clg,ad,so,we,ne,exp1,occ2,ind2,occ2_2,occ2_3,occ2_4,occ2_5,occ2_6,occ2_7,occ2_8,occ2_9,occ2_10,occ2_11,occ2_12,occ2_13,occ2_14,occ2_15,occ2_16,occ2_17,occ2_18,occ2_19,occ2_20,occ2_21,occ2_22,ind2_2,ind2_3,ind2_4,ind2_5,ind2_6,ind2_7,ind2_8,ind2_9,ind2_10,ind2_11,ind2_12,ind2_13,ind2_14,ind2_15,ind2_16,ind2_17,ind2_18,ind2_19,ind2_20,ind2_21,exp2,exp3,exp4,exp1_x_hsg,exp1_x_scl,exp1_x_clg,exp1_x_ad,exp1_x_so,exp1_x_we,exp1_x_ne,exp1_x_occ2,exp1_x_ind2,exp1_x_occ2_2,exp1_x_occ2_3,exp1_x_occ2_4,exp1_x_occ2_5,exp1_x_occ2_6,exp1_x_occ2_7,exp1_x_occ2_8,exp1_x_occ2_9,exp1_x_occ2_10,exp1_x_occ2_11,exp1_x_occ2_12,exp1_x_occ2_13,exp1_x_occ2_14,exp1_x_occ2_15,exp1_x_occ2_16,exp1_x_occ2_17,exp1_x_occ2_18,exp1_x_occ2_19,exp1_x_occ2_20,exp1_x_occ2_21,exp1_x_occ2_22,exp1_x_ind2_2,exp1_x_ind2_3,exp1_x_ind2_4,exp1_x_ind2_5,exp1_x_ind2_6,exp1_x_ind2_7,exp1_x_ind2_8,exp1_x_ind2_9,exp1_x_ind2_10,exp1_x_ind2_11,exp1_x_ind2_12,exp1_x_ind2_13,exp1_x_ind2_14,exp1_x_ind2_15,exp1_x_ind2_16,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,7.0,11,18,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.49,0.343,0.2401,0.0,0.0,7.0,0.0,0.0,0.0,7.0,77.0,126.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,31.0,10,9,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,9.61,29.791,92.3521,0.0,0.0,31.0,0.0,0.0,0.0,31.0,310.0,279.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,19,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.24,5.832,10.4976,18.0,0.0,0.0,0.0,0.0,0.0,18.0,342.0,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,25.0,1,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,6.25,15.625,39.0625,0.0,0.0,0.0,25.0,0.0,0.0,25.0,25.0,300.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,⋯
5,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,22.0,6,22,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.84,10.648,23.4256,0.0,0.0,22.0,0.0,0.0,0.0,22.0,132.0,484.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
6,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,5,14,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.01,0.001,0.0001,0.0,0.0,1.0,0.0,0.0,0.0,1.0,5.0,14.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,⋯
7,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,42.0,17,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,17.64,74.088,311.17,42.0,0.0,0.0,0.0,0.0,0.0,42.0,714.0,588.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

## 3. Linear Regressions

9. Split each of the dataframes created (basic, flexible and extra-flexible models) into a training sample (80% of the data) and a test sample.

In [93]:
# Combine the target 'Y' with the feature DataFrame for the basic model
data_basic = hcat(log_w, basic_model)

data_flex = hcat(log_w, flex_model)

data_extra_flex = hcat(log_w, extra_flex_model)

println("DataFrame for the basic model with the target:")
display(data_basic)

println("DataFrame for the flexible model with the target:")
display(data_flex)

println("DataFrame for the extra flexible model with the target:")
display(data_extra_flex)

DataFrame for the basic model with the target:


Row,lwage,sex,hsg,scl,clg,ad,so,we,ne,exp1,occ2,ind2,occ2_2,occ2_3,occ2_4,occ2_5,occ2_6,occ2_7,occ2_8,occ2_9,occ2_10,occ2_11,occ2_12,occ2_13,occ2_14,occ2_15,occ2_16,occ2_17,occ2_18,occ2_19,occ2_20,occ2_21,occ2_22,ind2_2,ind2_3,ind2_4,ind2_5,ind2_6,ind2_7,ind2_8,ind2_9,ind2_10,ind2_11,ind2_12,ind2_13,ind2_14,ind2_15,ind2_16,ind2_17,ind2_18,ind2_19,ind2_20,ind2_21,exp2,exp3,exp4,exp1_x_hsg,exp1_x_scl,exp1_x_clg,exp1_x_ad,exp1_x_so,exp1_x_we,exp1_x_ne,exp1_x_occ2,exp1_x_ind2,exp1_x_occ2_2,exp1_x_occ2_3,exp1_x_occ2_4,exp1_x_occ2_5,exp1_x_occ2_6,exp1_x_occ2_7,exp1_x_occ2_8,exp1_x_occ2_9,exp1_x_occ2_10,exp1_x_occ2_11,exp1_x_occ2_12,exp1_x_occ2_13,exp1_x_occ2_14,exp1_x_occ2_15,exp1_x_occ2_16,exp1_x_occ2_17,exp1_x_occ2_18,exp1_x_occ2_19,exp1_x_occ2_20,exp1_x_occ2_21,exp1_x_occ2_22,exp1_x_ind2_2,exp1_x_ind2_3,exp1_x_ind2_4,exp1_x_ind2_5,exp1_x_ind2_6,exp1_x_ind2_7,exp1_x_ind2_8,exp1_x_ind2_9,exp1_x_ind2_10,exp1_x_ind2_11,exp1_x_ind2_12,exp1_x_ind2_13,exp1_x_ind2_14,exp1_x_ind2_15,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,2.26336,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,7.0,11,18,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.49,0.343,0.2401,0.0,0.0,7.0,0.0,0.0,0.0,7.0,77.0,126.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
2,3.8728,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,31.0,10,9,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,9.61,29.791,92.3521,0.0,0.0,31.0,0.0,0.0,0.0,31.0,310.0,279.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
3,2.40313,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,19,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.24,5.832,10.4976,18.0,0.0,0.0,0.0,0.0,0.0,18.0,342.0,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
4,2.63493,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,25.0,1,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,6.25,15.625,39.0625,0.0,0.0,0.0,25.0,0.0,0.0,25.0,25.0,300.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,⋯
5,3.36198,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,22.0,6,22,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.84,10.648,23.4256,0.0,0.0,22.0,0.0,0.0,0.0,22.0,132.0,484.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
6,2.46222,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,5,14,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.01,0.001,0.0001,0.0,0.0,1.0,0.0,0.0,0.0,1.0,5.0,14.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,⋯
7,2.95651,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,42.0,17,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,17.64,74.088,311.17,42.0,0.0,0.0,0.0,0.0,0.0,42.0,714.0,588.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42.0,0.0,0.0,0.0,0.0,

DataFrame for the flexible model with the target:


Row,lwage,sex,hsg,scl,clg,ad,so,we,ne,exp1,occ2,ind2,occ2_2,occ2_3,occ2_4,occ2_5,occ2_6,occ2_7,occ2_8,occ2_9,occ2_10,occ2_11,occ2_12,occ2_13,occ2_14,occ2_15,occ2_16,occ2_17,occ2_18,occ2_19,occ2_20,occ2_21,occ2_22,ind2_2,ind2_3,ind2_4,ind2_5,ind2_6,ind2_7,ind2_8,ind2_9,ind2_10,ind2_11,ind2_12,ind2_13,ind2_14,ind2_15,ind2_16,ind2_17,ind2_18,ind2_19,ind2_20,ind2_21,exp2,exp3,exp4,exp1_x_hsg,exp1_x_scl,exp1_x_clg,exp1_x_ad,exp1_x_so,exp1_x_we,exp1_x_ne,exp1_x_occ2,exp1_x_ind2,exp1_x_occ2_2,exp1_x_occ2_3,exp1_x_occ2_4,exp1_x_occ2_5,exp1_x_occ2_6,exp1_x_occ2_7,exp1_x_occ2_8,exp1_x_occ2_9,exp1_x_occ2_10,exp1_x_occ2_11,exp1_x_occ2_12,exp1_x_occ2_13,exp1_x_occ2_14,exp1_x_occ2_15,exp1_x_occ2_16,exp1_x_occ2_17,exp1_x_occ2_18,exp1_x_occ2_19,exp1_x_occ2_20,exp1_x_occ2_21,exp1_x_occ2_22,exp1_x_ind2_2,exp1_x_ind2_3,exp1_x_ind2_4,exp1_x_ind2_5,exp1_x_ind2_6,exp1_x_ind2_7,exp1_x_ind2_8,exp1_x_ind2_9,exp1_x_ind2_10,exp1_x_ind2_11,exp1_x_ind2_12,exp1_x_ind2_13,exp1_x_ind2_14,exp1_x_ind2_15,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,2.26336,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,7.0,11,18,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.49,0.343,0.2401,0.0,0.0,7.0,0.0,0.0,0.0,7.0,77.0,126.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
2,3.8728,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,31.0,10,9,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,9.61,29.791,92.3521,0.0,0.0,31.0,0.0,0.0,0.0,31.0,310.0,279.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
3,2.40313,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,19,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.24,5.832,10.4976,18.0,0.0,0.0,0.0,0.0,0.0,18.0,342.0,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
4,2.63493,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,25.0,1,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,6.25,15.625,39.0625,0.0,0.0,0.0,25.0,0.0,0.0,25.0,25.0,300.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,⋯
5,3.36198,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,22.0,6,22,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.84,10.648,23.4256,0.0,0.0,22.0,0.0,0.0,0.0,22.0,132.0,484.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
6,2.46222,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,5,14,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.01,0.001,0.0001,0.0,0.0,1.0,0.0,0.0,0.0,1.0,5.0,14.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,⋯
7,2.95651,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,42.0,17,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,17.64,74.088,311.17,42.0,0.0,0.0,0.0,0.0,0.0,42.0,714.0,588.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42.0,0.0,0.0,0.0,0.0,

DataFrame for the extra flexible model with the target:


Row,lwage,sex,hsg,scl,clg,ad,so,we,ne,exp1,occ2,ind2,occ2_2,occ2_3,occ2_4,occ2_5,occ2_6,occ2_7,occ2_8,occ2_9,occ2_10,occ2_11,occ2_12,occ2_13,occ2_14,occ2_15,occ2_16,occ2_17,occ2_18,occ2_19,occ2_20,occ2_21,occ2_22,ind2_2,ind2_3,ind2_4,ind2_5,ind2_6,ind2_7,ind2_8,ind2_9,ind2_10,ind2_11,ind2_12,ind2_13,ind2_14,ind2_15,ind2_16,ind2_17,ind2_18,ind2_19,ind2_20,ind2_21,exp2,exp3,exp4,exp1_x_hsg,exp1_x_scl,exp1_x_clg,exp1_x_ad,exp1_x_so,exp1_x_we,exp1_x_ne,exp1_x_occ2,exp1_x_ind2,exp1_x_occ2_2,exp1_x_occ2_3,exp1_x_occ2_4,exp1_x_occ2_5,exp1_x_occ2_6,exp1_x_occ2_7,exp1_x_occ2_8,exp1_x_occ2_9,exp1_x_occ2_10,exp1_x_occ2_11,exp1_x_occ2_12,exp1_x_occ2_13,exp1_x_occ2_14,exp1_x_occ2_15,exp1_x_occ2_16,exp1_x_occ2_17,exp1_x_occ2_18,exp1_x_occ2_19,exp1_x_occ2_20,exp1_x_occ2_21,exp1_x_occ2_22,exp1_x_ind2_2,exp1_x_ind2_3,exp1_x_ind2_4,exp1_x_ind2_5,exp1_x_ind2_6,exp1_x_ind2_7,exp1_x_ind2_8,exp1_x_ind2_9,exp1_x_ind2_10,exp1_x_ind2_11,exp1_x_ind2_12,exp1_x_ind2_13,exp1_x_ind2_14,exp1_x_ind2_15,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,2.26336,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,7.0,11,18,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.49,0.343,0.2401,0.0,0.0,7.0,0.0,0.0,0.0,7.0,77.0,126.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
2,3.8728,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,31.0,10,9,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,9.61,29.791,92.3521,0.0,0.0,31.0,0.0,0.0,0.0,31.0,310.0,279.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
3,2.40313,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,19,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.24,5.832,10.4976,18.0,0.0,0.0,0.0,0.0,0.0,18.0,342.0,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
4,2.63493,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,25.0,1,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,6.25,15.625,39.0625,0.0,0.0,0.0,25.0,0.0,0.0,25.0,25.0,300.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,⋯
5,3.36198,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,22.0,6,22,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.84,10.648,23.4256,0.0,0.0,22.0,0.0,0.0,0.0,22.0,132.0,484.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
6,2.46222,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,5,14,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.01,0.001,0.0001,0.0,0.0,1.0,0.0,0.0,0.0,1.0,5.0,14.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,⋯
7,2.95651,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,42.0,17,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,17.64,74.088,311.17,42.0,0.0,0.0,0.0,0.0,0.0,42.0,714.0,588.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42.0,0.0,0.0,0.0,0.0,

In [94]:
using DataFrames, Random

# Set the seed for reproducibility
Random.seed!(123)

# Define a function to split a DataFrame into training and test sets
function split_train_test(df, train_ratio=0.8)
    # Shuffle the rows of the DataFrame randomly
    shuffled_df = shuffle(df)

    # Calculate the size of the training set
    n_train = round(Int, size(df, 1) * train_ratio)

    # Split the DataFrame into training and test sets
    train_df = shuffled_df[1:n_train, :]
    test_df = shuffled_df[(n_train+1):end, :]

    return train_df, test_df
end

# Apply the split function to each DataFrame
train_basic, test_basic = split_train_test(data_basic)
train_flex, test_flex = split_train_test(data_flex)
train_extra_flex, test_extra_flex = split_train_test(data_extra_flex)

# Display the results
println("Basic Model - Training Set:")
display(train_basic)

println("Basic Model - Test Set:")
display(test_basic)

println("Flexible Model - Training Set:")
display(train_flex)

println("Flexible Model - Test Set:")
display(test_flex)

println("Extra Flexible Model - Training Set:")
display(train_extra_flex)

println("Extra Flexible Model - Test Set:")
display(test_extra_flex)

Basic Model - Training Set:


Row,lwage,sex,hsg,scl,clg,ad,so,we,ne,exp1,occ2,ind2,occ2_2,occ2_3,occ2_4,occ2_5,occ2_6,occ2_7,occ2_8,occ2_9,occ2_10,occ2_11,occ2_12,occ2_13,occ2_14,occ2_15,occ2_16,occ2_17,occ2_18,occ2_19,occ2_20,occ2_21,occ2_22,ind2_2,ind2_3,ind2_4,ind2_5,ind2_6,ind2_7,ind2_8,ind2_9,ind2_10,ind2_11,ind2_12,ind2_13,ind2_14,ind2_15,ind2_16,ind2_17,ind2_18,ind2_19,ind2_20,ind2_21,exp2,exp3,exp4,exp1_x_hsg,exp1_x_scl,exp1_x_clg,exp1_x_ad,exp1_x_so,exp1_x_we,exp1_x_ne,exp1_x_occ2,exp1_x_ind2,exp1_x_occ2_2,exp1_x_occ2_3,exp1_x_occ2_4,exp1_x_occ2_5,exp1_x_occ2_6,exp1_x_occ2_7,exp1_x_occ2_8,exp1_x_occ2_9,exp1_x_occ2_10,exp1_x_occ2_11,exp1_x_occ2_12,exp1_x_occ2_13,exp1_x_occ2_14,exp1_x_occ2_15,exp1_x_occ2_16,exp1_x_occ2_17,exp1_x_occ2_18,exp1_x_occ2_19,exp1_x_occ2_20,exp1_x_occ2_21,exp1_x_occ2_22,exp1_x_ind2_2,exp1_x_ind2_3,exp1_x_ind2_4,exp1_x_ind2_5,exp1_x_ind2_6,exp1_x_ind2_7,exp1_x_ind2_8,exp1_x_ind2_9,exp1_x_ind2_10,exp1_x_ind2_11,exp1_x_ind2_12,exp1_x_ind2_13,exp1_x_ind2_14,exp1_x_ind2_15,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,2.76414,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,16.5,17,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2.7225,4.49212,7.41201,0.0,16.5,0.0,0.0,0.0,0.0,16.5,280.5,264.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
2,2.24502,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,25.0,13,20,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,6.25,15.625,39.0625,25.0,0.0,0.0,0.0,0.0,0.0,25.0,325.0,500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
3,1.10591,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,15,19,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.09,0.027,0.0081,0.0,3.0,0.0,0.0,0.0,0.0,0.0,45.0,57.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
4,3.53031,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,38.0,17,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,14.44,54.872,208.514,38.0,0.0,0.0,0.0,38.0,0.0,0.0,646.0,380.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38.0,0.0,0.0,0.0,0.0,0.0,⋯
5,2.95651,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,15.0,17,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2.25,3.375,5.0625,0.0,15.0,0.0,0.0,0.0,0.0,0.0,255.0,165.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,⋯
6,2.48651,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,3.0,17,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.09,0.027,0.0081,0.0,0.0,3.0,0.0,0.0,0.0,3.0,51.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,⋯
7,1.97568,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,7.0,16,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.49,0.343,0.2401,7.0,0.0,0.0,0.0,0.0,7.0,0.0,112.0,56.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.

Basic Model - Test Set:


Row,lwage,sex,hsg,scl,clg,ad,so,we,ne,exp1,occ2,ind2,occ2_2,occ2_3,occ2_4,occ2_5,occ2_6,occ2_7,occ2_8,occ2_9,occ2_10,occ2_11,occ2_12,occ2_13,occ2_14,occ2_15,occ2_16,occ2_17,occ2_18,occ2_19,occ2_20,occ2_21,occ2_22,ind2_2,ind2_3,ind2_4,ind2_5,ind2_6,ind2_7,ind2_8,ind2_9,ind2_10,ind2_11,ind2_12,ind2_13,ind2_14,ind2_15,ind2_16,ind2_17,ind2_18,ind2_19,ind2_20,ind2_21,exp2,exp3,exp4,exp1_x_hsg,exp1_x_scl,exp1_x_clg,exp1_x_ad,exp1_x_so,exp1_x_we,exp1_x_ne,exp1_x_occ2,exp1_x_ind2,exp1_x_occ2_2,exp1_x_occ2_3,exp1_x_occ2_4,exp1_x_occ2_5,exp1_x_occ2_6,exp1_x_occ2_7,exp1_x_occ2_8,exp1_x_occ2_9,exp1_x_occ2_10,exp1_x_occ2_11,exp1_x_occ2_12,exp1_x_occ2_13,exp1_x_occ2_14,exp1_x_occ2_15,exp1_x_occ2_16,exp1_x_occ2_17,exp1_x_occ2_18,exp1_x_occ2_19,exp1_x_occ2_20,exp1_x_occ2_21,exp1_x_occ2_22,exp1_x_ind2_2,exp1_x_ind2_3,exp1_x_ind2_4,exp1_x_ind2_5,exp1_x_ind2_6,exp1_x_ind2_7,exp1_x_ind2_8,exp1_x_ind2_9,exp1_x_ind2_10,exp1_x_ind2_11,exp1_x_ind2_12,exp1_x_ind2_13,exp1_x_ind2_14,exp1_x_ind2_15,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,2.59984,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,4.5,11,16,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.2025,0.091125,0.0410063,0.0,4.5,0.0,0.0,0.0,4.5,0.0,49.5,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
2,3.27497,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,4.0,2,12,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.16,0.064,0.0256,0.0,0.0,4.0,0.0,0.0,0.0,4.0,8.0,48.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,⋯
3,3.03655,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,21,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.76,17.576,45.6976,26.0,0.0,0.0,0.0,0.0,0.0,0.0,546.0,156.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,0.0,0.0,0.0,0.0,0.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
4,3.51613,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,21,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
5,3.5717,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,15.0,20,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2.25,3.375,5.0625,0.0,0.0,15.0,0.0,0.0,0.0,15.0,300.0,195.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,⋯
6,3.05182,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,15,19,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.49,0.343,0.2401,0.0,0.0,0.0,0.0,0.0,0.0,0.0,105.0,133.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
7,2.30508,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,8,17,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

Flexible Model - Training Set:


...

Row,lwage,sex,hsg,scl,clg,ad,so,we,ne,exp1,occ2,ind2,occ2_2,occ2_3,occ2_4,occ2_5,occ2_6,occ2_7,occ2_8,occ2_9,occ2_10,occ2_11,occ2_12,occ2_13,occ2_14,occ2_15,occ2_16,occ2_17,occ2_18,occ2_19,occ2_20,occ2_21,occ2_22,ind2_2,ind2_3,ind2_4,ind2_5,ind2_6,ind2_7,ind2_8,ind2_9,ind2_10,ind2_11,ind2_12,ind2_13,ind2_14,ind2_15,ind2_16,ind2_17,ind2_18,ind2_19,ind2_20,ind2_21,exp2,exp3,exp4,exp1_x_hsg,exp1_x_scl,exp1_x_clg,exp1_x_ad,exp1_x_so,exp1_x_we,exp1_x_ne,exp1_x_occ2,exp1_x_ind2,exp1_x_occ2_2,exp1_x_occ2_3,exp1_x_occ2_4,exp1_x_occ2_5,exp1_x_occ2_6,exp1_x_occ2_7,exp1_x_occ2_8,exp1_x_occ2_9,exp1_x_occ2_10,exp1_x_occ2_11,exp1_x_occ2_12,exp1_x_occ2_13,exp1_x_occ2_14,exp1_x_occ2_15,exp1_x_occ2_16,exp1_x_occ2_17,exp1_x_occ2_18,exp1_x_occ2_19,exp1_x_occ2_20,exp1_x_occ2_21,exp1_x_occ2_22,exp1_x_ind2_2,exp1_x_ind2_3,exp1_x_ind2_4,exp1_x_ind2_5,exp1_x_ind2_6,exp1_x_ind2_7,exp1_x_ind2_8,exp1_x_ind2_9,exp1_x_ind2_10,exp1_x_ind2_11,exp1_x_ind2_12,exp1_x_ind2_13,exp1_x_ind2_14,exp1_x_ind2_15,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,2.31215,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,10.5,14,13,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.1025,1.15762,1.21551,0.0,10.5,0.0,0.0,0.0,10.5,0.0,147.0,136.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.5,0.0,0.0,⋯
2,3.27497,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6.0,4,14,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.36,0.216,0.1296,0.0,6.0,0.0,0.0,0.0,0.0,0.0,24.0,84.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,⋯
3,3.17966,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.01,0.001,0.0001,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
4,2.95651,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,8,17,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.01,0.001,0.0001,0.0,0.0,0.0,1.0,0.0,1.0,0.0,8.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
5,3.05182,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,20.5,22,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4.2025,8.61513,17.661,0.0,20.5,0.0,0.0,0.0,20.5,0.0,451.0,164.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.5,0.0,0.0,0.0,0.0,0.0,0.0,20.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
6,4.17291,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,31.0,2,12,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,9.61,29.791,92.3521,0.0,0.0,31.0,0.0,31.0,0.0,0.0,62.0,372.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,⋯
7,2.82298,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,5,18,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

Extra Flexible Model - Training Set:


Row,lwage,sex,hsg,scl,clg,ad,so,we,ne,exp1,occ2,ind2,occ2_2,occ2_3,occ2_4,occ2_5,occ2_6,occ2_7,occ2_8,occ2_9,occ2_10,occ2_11,occ2_12,occ2_13,occ2_14,occ2_15,occ2_16,occ2_17,occ2_18,occ2_19,occ2_20,occ2_21,occ2_22,ind2_2,ind2_3,ind2_4,ind2_5,ind2_6,ind2_7,ind2_8,ind2_9,ind2_10,ind2_11,ind2_12,ind2_13,ind2_14,ind2_15,ind2_16,ind2_17,ind2_18,ind2_19,ind2_20,ind2_21,exp2,exp3,exp4,exp1_x_hsg,exp1_x_scl,exp1_x_clg,exp1_x_ad,exp1_x_so,exp1_x_we,exp1_x_ne,exp1_x_occ2,exp1_x_ind2,exp1_x_occ2_2,exp1_x_occ2_3,exp1_x_occ2_4,exp1_x_occ2_5,exp1_x_occ2_6,exp1_x_occ2_7,exp1_x_occ2_8,exp1_x_occ2_9,exp1_x_occ2_10,exp1_x_occ2_11,exp1_x_occ2_12,exp1_x_occ2_13,exp1_x_occ2_14,exp1_x_occ2_15,exp1_x_occ2_16,exp1_x_occ2_17,exp1_x_occ2_18,exp1_x_occ2_19,exp1_x_occ2_20,exp1_x_occ2_21,exp1_x_occ2_22,exp1_x_ind2_2,exp1_x_ind2_3,exp1_x_ind2_4,exp1_x_ind2_5,exp1_x_ind2_6,exp1_x_ind2_7,exp1_x_ind2_8,exp1_x_ind2_9,exp1_x_ind2_10,exp1_x_ind2_11,exp1_x_ind2_12,exp1_x_ind2_13,exp1_x_ind2_14,exp1_x_ind2_15,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,2.47554,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,1,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.81,0.729,0.6561,9.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,180.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
2,2.95651,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,8.0,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.64,0.512,0.4096,0.0,8.0,0.0,0.0,8.0,0.0,0.0,16.0,16.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
3,3.07429,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,3.0,3,14,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.09,0.027,0.0081,0.0,0.0,3.0,0.0,0.0,3.0,0.0,9.0,42.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,⋯
4,2.158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.5,13,20,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1.3225,1.52087,1.74901,0.0,0.0,0.0,0.0,0.0,0.0,0.0,149.5,230.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
5,3.46734,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,13.0,19,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.69,2.197,2.8561,0.0,13.0,0.0,0.0,0.0,0.0,0.0,247.0,78.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
6,1.34707,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,7.5,16,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.5625,0.421875,0.316406,0.0,7.5,0.0,0.0,0.0,7.5,0.0,120.0,67.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.5,0.0,0.0,0.0,0.0,0.0,0.0,⋯
7,2.29615,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,7.0,1,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.49,0.343,0.2401,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,63.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

Extra Flexible Model - Test Set:


Row,lwage,sex,hsg,scl,clg,ad,so,we,ne,exp1,occ2,ind2,occ2_2,occ2_3,occ2_4,occ2_5,occ2_6,occ2_7,occ2_8,occ2_9,occ2_10,occ2_11,occ2_12,occ2_13,occ2_14,occ2_15,occ2_16,occ2_17,occ2_18,occ2_19,occ2_20,occ2_21,occ2_22,ind2_2,ind2_3,ind2_4,ind2_5,ind2_6,ind2_7,ind2_8,ind2_9,ind2_10,ind2_11,ind2_12,ind2_13,ind2_14,ind2_15,ind2_16,ind2_17,ind2_18,ind2_19,ind2_20,ind2_21,exp2,exp3,exp4,exp1_x_hsg,exp1_x_scl,exp1_x_clg,exp1_x_ad,exp1_x_so,exp1_x_we,exp1_x_ne,exp1_x_occ2,exp1_x_ind2,exp1_x_occ2_2,exp1_x_occ2_3,exp1_x_occ2_4,exp1_x_occ2_5,exp1_x_occ2_6,exp1_x_occ2_7,exp1_x_occ2_8,exp1_x_occ2_9,exp1_x_occ2_10,exp1_x_occ2_11,exp1_x_occ2_12,exp1_x_occ2_13,exp1_x_occ2_14,exp1_x_occ2_15,exp1_x_occ2_16,exp1_x_occ2_17,exp1_x_occ2_18,exp1_x_occ2_19,exp1_x_occ2_20,exp1_x_occ2_21,exp1_x_occ2_22,exp1_x_ind2_2,exp1_x_ind2_3,exp1_x_ind2_4,exp1_x_ind2_5,exp1_x_ind2_6,exp1_x_ind2_7,exp1_x_ind2_8,exp1_x_ind2_9,exp1_x_ind2_10,exp1_x_ind2_11,exp1_x_ind2_12,exp1_x_ind2_13,exp1_x_ind2_14,exp1_x_ind2_15,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,3.36198,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,5.0,9,14,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.25,0.125,0.0625,0.0,0.0,5.0,0.0,5.0,0.0,0.0,45.0,70.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,⋯
2,1.75254,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,10.5,16,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1.1025,1.15762,1.21551,0.0,10.5,0.0,0.0,10.5,0.0,0.0,168.0,94.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.5,0.0,0.0,0.0,0.0,0.0,0.0,⋯
3,2.30259,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,17,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.76,13.824,33.1776,24.0,0.0,0.0,0.0,0.0,0.0,0.0,408.0,120.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
4,2.44569,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,7.0,22,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.49,0.343,0.2401,7.0,0.0,0.0,0.0,7.0,0.0,0.0,154.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
5,2.57922,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,17,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.36,0.216,0.1296,6.0,0.0,0.0,0.0,0.0,0.0,0.0,102.0,120.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
6,2.59984,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,13.0,3,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.69,2.197,2.8561,0.0,0.0,13.0,0.0,0.0,0.0,0.0,39.0,65.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
7,2.85115,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,12.5,10,18,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1.5625,1.95312,2.44141,0.0,12.5,0.0,0.0,0.0,0.0,12.5,125.0,225.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

10. Estimate all three models

In [95]:
# Se define una función para crear la fórmula dinámicamente usando todas las características excepto la target para crear un modelo que considere todas las otras variables
function create_formula(data::DataFrame, target::Symbol)
    # Ensure the target variable is not included in the features
    data_features = select(data, Not(target))  # Remove the target column from the DataFrame

    # Get all column names except the target
    features = names(data_features)  # Now, features will only contain independent variables

    # Convert the feature names to symbols
    features_symbols = Symbol.(features)

    # Create the formula using the terms as symbols
    return Term(target) ~ sum(Term.(features_symbols))
end

create_formula (generic function with 1 method)

In [96]:
# Create formulas for each model
formula_basic = create_formula(data_basic, :lwage)
formula_flex = create_formula(data_flex, :lwage)
formula_extra_flex = create_formula(data_extra_flex, :lwage)

# Perform OLS estimation for each model

# OLS estimation for the basic model
ols_basic = lm(formula_basic, data_basic)

# OLS estimation for the flexible model
ols_flex = lm(formula_flex, data_flex)

# OLS estimation for the extra flexible model
ols_extra_flex = lm(formula_extra_flex, data_extra_flex)

# Display a summary of the results for each model
function display_summary(model, model_name)
    println("Summary of OLS results for ", model_name)
    println(coeftable(model))  # Display the coefficient table
    println("R-squared: ", r2(model))  # Display the R-squared value
    println("\n")
end

# Display the results for each model
display_summary(ols_basic, "Basic Model")
display_summary(ols_flex, "Flexible Model")
display_summary(ols_extra_flex, "Extra Flexible Model")

Summary of OLS results for Basic Model
────────────────────────────────────────────────────────────────────────────────────────────
                          Coef.    Std. Error       t  Pr(>|t|)     Lower 95%      Upper 95%
────────────────────────────────────────────────────────────────────────────────────────────
(Intercept)         0.0          NaN           NaN       NaN     NaN           NaN
sex                -0.0612705      0.0159811    -3.83    0.0001   -0.0926015    -0.0299395
hsg                -0.285739       1.29771      -0.22    0.8257   -2.82991       2.25844
scl                 0.0          NaN           NaN       NaN     NaN           NaN
clg                 1.05987        1.35214       0.78    0.4332   -1.59101       3.71076
ad                  0.0          NaN           NaN       NaN     NaN           NaN
so                 -0.110469       0.331538     -0.33    0.7390   -0.760452      0.539514
we                  0.483125       0.386649      1.25    0.2115   -0.27490

11. Report the MSE and the R2 for both samples of each model, and the adjusted R2 for the training sample. Do we see overfitting? Which model has the best out-of-sample performance? For reference, you should report the following:

In [97]:
# Function to calculate Mean Squared Error (MSE)
function compute_mse(model, data)
    # Predict the values using the model
    predictions = predict(model, data)
    # Calculate the residuals
    residuals = data.lwage .- predictions
    # Compute MSE
    mse = mean(residuals .^ 2)
    return mse
end

# Function to compute Adjusted R-squared
function adjusted_r2(model, data)
    r2_val = r2(model)  # Compute R-squared
    n = nrow(data)  # Number of observations
    p = length(coef(model)) - 1  # Number of predictors
    adj_r2 = 1 - (1 - r2_val) * (n - 1) / (n - p - 1)  # Adjusted R-squared formula
    return adj_r2
end

# Compute metrics for each model
function report_metrics(formula, train_data, test_data, model_name)
    # Fit the model using the training data
    model_train = lm(formula, train_data)

    # Compute MSE for training and test samples
    mse_train = compute_mse(model_train, train_data)
    mse_test = compute_mse(model_train, test_data)

    # Compute R-squared for training and test samples
    r2_train = r2(model_train)
    r2_test = r2(lm(formula, test_data))

    # Compute Adjusted R-squared for training sample only
    adj_r2_train = adjusted_r2(model_train, train_data)

    # Print the results
    println("Metrics for ", model_name, ":")
    println("MSE (Training Sample): ", mse_train)
    println("MSE (Test Sample): ", mse_test)
    println("R-squared (Training Sample): ", r2_train)
    println("R-squared (Test Sample): ", r2_test)
    println("Adjusted R-squared (Training Sample): ", adj_r2_train)
    println("\n")
end

# Assuming the formulas have been created correctly
# Call the function to report metrics for each model
report_metrics(formula_basic, train_basic, test_basic, "Model 1")
report_metrics(formula_flex, train_flex, test_flex, "Model 2")
report_metrics(formula_extra_flex, train_extra_flex, test_extra_flex, "Model 3")

Metrics for Model 1:
MSE (Training Sample): 0.17403285184551884
MSE (Test Sample): 0.27236082702913966
R-squared (Training Sample): 0.4626245563385588
R-squared (Test Sample): 0.7464137425032822
Adjusted R-squared (Training Sample): 0.15934316276434624


Metrics for Model 2:
MSE (Training Sample): 0.17368003242361332
MSE (Test Sample): 0.2644543886971121
R-squared (Training Sample): 0.47197015979010004
R-squared (Test Sample): 0.7413903023914457
Adjusted R-squared (Training Sample): 0.1739631933822341


Metrics for Model 3:
MSE (Training Sample): 0.16969382964626156
MSE (Test Sample): 0.47391382081674177
R-squared (Training Sample): 0.4723386984801222
R-squared (Test Sample): 0.7282604253252403
Adjusted R-squared (Training Sample): 0.17453972618291813




Do We See Overfitting?


*   Model 1: The training and test MSEs are reasonably close, and the test R² is higher than the training R², indicating that Model 1 generalizes well and is likely not overfitting.
*  Model 2: Similar behavior to Model 1 with slightly lower MSE and better adjusted R², suggesting it might be slightly better than Model 1 but not overfitting.
*    Model 3: The test MSE is substantially larger than the training MSE, which suggests that this model is overfitting. It performs well on the training set but poorly on the test set.


Best Out-of-Sample Performance:


Model 2 seems to have the best out-of-sample performance because:
Its MSE on the test set is the lowest (0.264).
Its R² (Test Sample) is slightly lower than Model 1 but quite close.
Its adjusted R² is the highest, suggesting a better balance between fit and complexity.


Conclusion:

Model 2 appears to offer the best trade-off between complexity and generalization based on the metrics provided.

Model 3 shows signs of overfitting because of the large disparity between training and test MSE.

Model 1 is a decent model but slightly underperforms compared to Model 2 based on test MSE.


